https://adventofcode.com/2024/day/6

In [127]:
from helpers import *
import re
import copy

In [128]:
lines = read_lines("input-06.txt")

## Part 1

First find the guard. This gives us the row and column of the guards position and the character representing them so we know the starting direction.

In [129]:
for row in range(0, len(lines)):
    m = re.search("[<>v^]", lines[row])
    if m:
        col = m.span()[0]
        guard = m.group()
        break
pos = (row, col)

if guard == "^":
    dir = (-1, 0)
elif guard == "v":
    dir = (1, 0)
elif guard == ">":
    dir = (0, 1)
elif guard == "<":
    dir = (-1, 0)
else:
    print("Don't recognise this guard!")

lab = []
for l in lines:
    lab.append([c for c in l])    
lab[row][col] = 'X'

Now to wander around the lab following the rules until we escape, counting steps as we go.

In [150]:
def escaped(p):
    return p[0] < 0 or p[1] < 0 or p[0] >= len(lines) or p[1] >= len(lines)

def patrol_route(start, dir, lab, limit):
    count = 1
    route = []
    loop_detect = {}
    pos = start
    next_pos = (pos[0] + dir[0], pos[1] + dir[1])
    
    while not escaped(next_pos):
        next_square = lab[next_pos[0]][next_pos[1]]
        #print(pos)
        if next_square != "#":
            pos = next_pos

            if (pos[0], pos[1]) in loop_detect:
                if dir in loop_detect[(pos[0], pos[1])]:
                    if count > 5000:
                        print(count)
                    return []
                else:
                    loop_detect[(pos[0], pos[1])].append(dir)
            else:
                loop_detect[(pos[0], pos[1])] = [dir]            
                
            next_pos = (pos[0] + dir[0], pos[1] + dir[1])
            count += 1
            if count > limit:
                print("Hit the loop limit - shouldn't happen?")
                return []
            if next_square == '.':
                lab[pos[0]][pos[1]] = 'X'
                route.append((pos[0], pos[1]))

        else:
            dir = (dir[1], dir[0]*-1) # Rotate right 90
            next_pos = (pos[0] + dir[0], pos[1] + dir[1])
    
    return route


In [139]:
route = patrol_route(pos, dir, copy.deepcopy(lab), 10000)

print(len(route)+1)

5305


## Part two

Now to look for places for obstacles that would create a loop.

Not sure how to go about this, other than brute force. I guess there's only a point in trying places on the unobstructed route previously found - the guard is not going to visit any other places. So, I'll modify the above code to keep a list of those places and then here we can just try them all.

The next problem is how to detect a loop. For now, just counting steps and assuming a loop if we hit a limit. Most loops will be much shorter than that, I assume, and so something more intelligent might well be faster. Running on my Pi 4 with this simple loop detection, the whole process takes about 3 minutes. Long enough to be annoying but maybe not long enough to bother trying to optimise it! Perhaps if I find some enthusiasm somewhere...

In [151]:
%%time

loop_count = 0

for p in route:
    #print("Checking:", p)
    lab2 = copy.deepcopy(lab)
    lab2[p[0]][p[1]] = "#"
    new_route = patrol_route(pos, dir, lab2, 10000)
    if len(new_route) == 0:
        loop_count += 1

print(loop_count)

5012
5555
5431
5654
5454
5233
5103
5040
5248
5007
5028
5012
5081
5086
5007
5052
5051
5055
5022
5078
5065
5150
5036
5094
5063
5044
5130
5078
5132
5068
5120
5056
5093
5065
5149
5136
5186
5094
5154
5158
5083
5164
5078
5137
5163
5139
5104
5098
5101
5117
5118
5174
5124
5230
5132
5114
5117
5217
5219
5142
5145
5238
5128
5130
5143
5247
5216
5196
5141
5144
5165
5172
5246
5270
5163
5183
5154
5157
5159
5161
5235
5239
5236
5171
5213
5191
5192
5324
5181
5215
5289
5237
5236
5219
5338
5297
5574
5206
5344
5267
5307
5254
5311
5225
5212
5272
5228
5276
5464
5217
5235
5220
5229
5223
5282
5334
5253
5331
5232
5253
5241
5303
5300
5239
5346
5253
5279
5272
5750
5250
5417
5268
5286
5299
5258
5280
5308
5389
5281
5265
5301
5292
5305
5299
5363
5284
5322
5284
5307
5313
5318
5312
5323
5305
5383
5428
5324
5314
5325
5321
5326
5397
5468
5330
5332
5401
5335
5337
5340
5526
5345
5374
5397
5426
5355
5374
5870
5356
5359
5432
5375
5437
5434
5370
5371
5480
5387
5396
5413
5386
5468
5378
5551
5402
5416
5420
5433
5386
5414
5442


2143

In [137]:
loop_count

202